# Лабораторная работа №3. Методы поиска подстроки в строке
## Выполнил студент группы БВТ2003 Гореликов Владислав Павлович
# Задание 1 
Реализовать методы поиска подстроки в строке. Добавить возможность
ввода строки и подстроки с клавиатуры. Предусмотреть возможность
существования пробела. Реализовать возможность выбора опции
чувствительности или нечувствительности к регистру. Оценить время работы
каждого алгоритма поиска и сравнить его со временем работы стандартной
функции поиска, используемой в выбранном языке программирования. 
Алгоритмы: 
1.Кнута-Морриса-Пратта
2.Упрощенный Бойера-Мура

In [5]:
%%time
# Префикс-функция(пи-массив)
def prefix(st):
    pi = [0] * len(st)
    i, j = 0, 1
    while j < len(st):
        if st[i] == st[j]:
            pi[j] = i + 1
            i += 1
            j += 1
        elif i:
            i = pi[i - 1]
        else:
            pi[j] = 0
            j += 1
    return pi

#Алгоритм Кнута-Морриса-Пратта
print('КМП')
print('Строка: ')
st = input()
print('Подстрока: ')
post = input()
def KMP(st, post):

    s1 = st.replace(" ", "")
    w1 = post.replace(" ", "")

    s = s1.lower()
    w = w1.lower()

    n = 0
    R = []
    pi = prefix(s)

    for i in range(len(s)):
        while n > 0 and s[i] != w[n]:
            n = pi[n - 1]
        if s[i] == w[n]:
            n += 1
        if n == len(w):
            R.append((i - len(w) + 1, i + 1))
            n = pi[n - 1]

    if R != []:
        return R
    else: return 'Not found'

print("Кнута-Морриса-Пратта: ", KMP(st, post))

КМП
Строка: 
hellohelp
Подстрока: 
help
Кнута-Морриса-Пратта:  [(5, 9)]
Wall time: 9.33 s


In [10]:
%%time
# таблица смещения
def tabsm(word):
    tabsm = [len(word)] * 256
    for i in range(len(word) - 1):
        tabsm[ord(word[i])] = len(word) - 1 - i
    return tabsm

# алгоритм бойера-мура
print('БМ')
print('Строка: ')
str = input()
print('Подстрока: ')
word = input()
def BoierMur(str, word):

    s = str.lower()
    w = word.lower()
    s = s.replace(' ', '')
    w = w.replace(' ', '')

    R2 = []
    tab = tabsm(w)
    skip = 0
    while len(s) - skip >= len(w):
        if s[skip:skip + len(w)] == w:
            R2.append((skip, skip + len(w)))
        skip += tab[ord(s[skip + len(w) - 1])]
    if R2 != []:
        return R2
    else: return 'Not found'
print("Упрощенный Бойера-Мура: ", BoierMur(str, word))

БМ
Строка: 
hellohelp
Подстрока: 
help
Упрощенный Бойера-Мура:  [(5, 9)]
Wall time: 4.79 s


# Задание 2 «Пятнашки» 
Игра в 15, пятнашки, такен — популярная головоломка, придуманная
в 1878 году Ноем Чепмэном. Она представляет собой набор
одинаковых квадратных костяшек с нанесёнными числами, заключённых в
квадратную коробку. Длина стороны коробки в четыре раза больше длины
стороны костяшек для набора из 15 элементов, соответственно в коробке
остаётся незаполненным одно квадратное поле. Цель игры — перемещая
костяшки по коробке, добиться упорядочивания их по номерам, желательно
сделав как можно меньше перемещений.

Задача: написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке. 

Входные данные: массив чисел, представляющий собой расстановку в
порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле. 
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов. 

Выходные данные: если решения нет, то функция должна вернуть
пустой массив []. Если решение есть, то необходимо представить решение — 
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

Например, для начального расположения элементов [1, 2, 3, 4, 5, 6, 7, 8, 
13, 9, 11, 12, 10, 14, 15, 0] одним из возможных решений будет [15, 14, 10, 13, 
9, 10, 14, 15] (последовательность шагов здесь: двигаем 15, двигаем 14, 
двигаем 10, …, двигаем 15)

In [1]:
from queue import PriorityQueue

class Position:
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    def __lt__(self, other):
        return self.start_distance < other.start_distance

    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

N = 4

def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     # вниз
    if i < N - 1: offsets.append(N)  # вверх
    if j > 0: offsets.append(-1)     # вправо
    if j < N - 1: offsets.append(1)  # влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))


def parity(permutation):
    seen, cycles = set(), 0
    for i in permutation:
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2


SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

if parity(start) == 0:
    print('Нерешаемо')
else:
    start = tuple(start)
    p = Position(start, 0)
    print(p, '\n')

    candidates = PriorityQueue()
    candidates.put(p)

    visited = set([p])
    came_from = {p.position: None}
    
    while p.position != SOLVED:
        p = candidates.get()
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    path = []
    prev = p
    while p.position != start:
        #запоминаем откуда был ход
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0 

[15, 14, 10, 13, 9, 10, 14, 15]


## Вывод: 
В данной лараторной работе мы познакомились и научились работать с двумя основными методами поиска подстроки в строке